# CAO Points Analysis

***

In [1]:
# Regular expressions
import re

# Convenient HTTP request.
import requests as rq

# Dates and Times
import datetime as dt

# Data Frame
import pandas as pd

# For downloading
import urllib.request as urlrq

In [2]:
# Get the current data and time
now = dt.datetime.now()

# Format as a string.
nowstr = now.strftime('%y%m%d_%H%M%S')

<br>

## 2021 CAO POINTS

http://www2.cao.ie/points/l8.php

***

In [3]:
# Fetch the CAO points url
resp = rq.get('http://www2.cao.ie/points/l8.php')
resp

<Response [200]>

# Save Original Data Set

***

In [4]:
# Create a file path for the original data
pathhtml = 'data/cao2021_' + nowstr + '.html'

# Error on Server

Technically, the server says we should decode as per:

'''
Content-Type: text/html; charset=iso-8859-1
'''

However, one line uses \x96 which isn't defined in iso-8859-1.

Therefore we use the similar decoding standard cp1252, which is very similar but includes #x96.

***

In [5]:
# The server uses the wrong encoding, fix it
original_encoding = resp.encoding

#Change to cp1252
resp.encoding = 'cp1252'

In [6]:
# Save the orignal html file
with open(pathhtml, 'w') as f:
    f.write(resp.text)

## Use regular expressions to slect lines we want

In [7]:
# Compile the regular expression for matching lines
re_course = re.compile(r'([A-Z]{2}[0-9]{3})(.*)')

### Loop through the lines of the repsonse

In [8]:
# The file path for the csv file
path2021 = 'data/cao2021_csv_' + nowstr + '.csv'

In [9]:
# Keep track of how many courses we process
no_lines = 0

# Open the csv file for writing
with open(path2021, 'w') as f:
    # Write a header row
    f.write(','.join(['code', 'title', 'pointsR1', 'pointsR2']) + '\n')
    # Loop through the lines of the response content
    for line in resp.iter_lines():
        #Decode the line, using the wrong encoding
        dline = line.decode('cp1252')
        # Match only the lines we want - the ones representing courses.
        if re_course.fullmatch(dline):
            #Add one to the lines counter
            no_lines = no_lines + 1
            #The course code
            course_code = dline[:5]
            #The course Title
            course_title = dline[7:57].strip()
            # Round one points
            course_points = re.split(' +', dline[60:])
            if len(course_points) != 2:
                course_points = course_points[:2]
            #join the fields using a comma
            linesplit = [course_code, course_title, course_points[0], course_points[1]]
           
            #Rejoin the substrings with commas inbetween
            f.write(','.join(linesplit) + '\n')
 # print      
print(f"Total number of lines is {no_lines}.")

Total number of lines is 949.


#### NB: It was verified as of 03/11/2021 that there were 949 courses exactly in the CA 2021 points list

In [10]:
df2021 = pd.read_csv(path2021, encoding='cp1252')

In [11]:
df2021

code                                              title pointsR1  \
0    AL801     Software Design for Virtual Reality and Gaming      300   
1    AL802  Software Design in Artificial Intelligence for...      313   
2    AL803  Software Design for Mobile Apps and Connected ...      350   
3    AL805    Computer Engineering for Network Infrastructure      321   
4    AL810                                 Quantity Surveying      328   
..     ...                                                ...      ...   
944  WD211                                 Creative Computing      270   
945  WD212                    Recreation and Sport Management      262   
946  WD230           Mechanical and Manufacturing Engineering      230   
947  WD231                 Early Childhood Care and Education      266   
948  WD232                       Business Information Systems      261   

    pointsR2  
0        NaN  
1        NaN  
2        NaN  
3        NaN  
4        NaN  
..       ...  
944      NaN  
945      NaN  
946      230  
947      NaN  
948      261  

[949 rows x 4 columns]

<br>

## 2020 points

http://www.cao.ie/index.php?page=points&p=2020

***

In [12]:
url2020 = 'http://www2.cao.ie/points/CAOPointsCharts2020.xlsx'

### Save Original File

In [13]:
# Create a file path for the original data
pathxlsx = 'data/cao2020_' + nowstr + '.xlsx'

In [14]:
urlrq.urlretrieve(url2020, pathxlsx)

('data/cao2020_211110_202919.xlsx', <http.client.HTTPMessage at 0x1816a21b460>)

<br>

#### Load Spreadsheet using pandas

***

In [15]:
# Download and parse the Excel spreadsheet
df2020 = pd.read_excel(url2020, skiprows=10)

In [16]:
df2020 ['COURSE CODE2'].drop_duplicates()

0       AC120
1       AC137
2       AD101
3       AD102
4       AD103
        ...  
1459    WD208
1460    WD210
1461    WD211
1462    WD212
1463    WD230
Name: COURSE CODE2, Length: 1464, dtype: object

In [17]:
# spot check random row
df2020.iloc[753]

CATEGORY (i.e.ISCED description)          Engineering and engineering trades
COURSE TITLE                        Road Transport Technology and Management
COURSE CODE2                                                           LC286
R1 POINTS                                                                264
R1 Random *                                                              NaN
R2 POINTS                                                                NaN
R2 Random*                                                               NaN
EOS                                                                      264
EOS Random *                                                             NaN
EOS Mid-point                                                            360
LEVEL                                                                      7
HEI                                         Limerick Institute of Technology
Test/Interview #                                                         NaN

In [18]:
# spot check last row
df2020.iloc[-1]

CATEGORY (i.e.ISCED description)          Engineering and engineering trades
COURSE TITLE                        Mechanical and Manufacturing Engineering
COURSE CODE2                                                           WD230
R1 POINTS                                                                253
R1 Random *                                                              NaN
R2 POINTS                                                                NaN
R2 Random*                                                               NaN
EOS                                                                      253
EOS Random *                                                             NaN
EOS Mid-point                                                            369
LEVEL                                                                      8
HEI                                        Waterford Institute of Technology
Test/Interview #                                                         NaN

In [19]:
# Create a file path for the panda data.
path2020 = 'data/cao2020_' + nowstr + '.csv'

In [20]:
# Save pandas data frame to disk
df2020.to_csv(path2020)

<br>

## 2019 points

http://www2.cao.ie/points/lvl8_19.pdf

***

In [21]:
import urllib.request
import tabula

url2019 = 'http://www2.cao.ie/points/lvl8_19.pdf'
path = "data/cao2019_"+nowstr+".csv"

In [22]:
# Read pdf into a list of DataFrame 
# dfs = tabula.read_pdf(path, pages='all')
tabula.convert_into("http://www2.cao.ie/points/lvl8_19.pdf", path, output_format="csv", pages='all')

In [23]:
df2019 = pd.read_csv(path, encoding='cp1252')

In [24]:
df2=df.DataFrame(df2019[df2019['Course Code'].isnull()]['INSTITUTION and COURSE'].astype(str))
df1 = df.DataFrame(df2019['Course Code'].astype(str).str[:2].unique().astype(str)[1:])
df3 = pd.concat([df1, df2])
df3
#df_2019_collegenames =  pd.concat(frames)
#df_2019_collegenames

NameError: name 'df' is not defined

In [ ]:
df2019 = pd.read_csv('data/cao2019_20211029_113930_edited.csv', sep='\t')

In [ ]:
df2019

</br>

## Concat and Join

***

In [25]:
courses2021 = df2021[['code', 'title']]
courses2021

code                                              title
0    AL801     Software Design for Virtual Reality and Gaming
1    AL802  Software Design in Artificial Intelligence for...
2    AL803  Software Design for Mobile Apps and Connected ...
3    AL805    Computer Engineering for Network Infrastructure
4    AL810                                 Quantity Surveying
..     ...                                                ...
944  WD211                                 Creative Computing
945  WD212                    Recreation and Sport Management
946  WD230           Mechanical and Manufacturing Engineering
947  WD231                 Early Childhood Care and Education
948  WD232                       Business Information Systems

[949 rows x 2 columns]

In [26]:
courses2020 = df2020[['COURSE CODE2', 'COURSE TITLE']]
courses2020

COURSE CODE2                                       COURSE TITLE
0           AC120                             International Business
1           AC137                                       Liberal Arts
2           AD101   First Year Art & Design (Common Entry,portfolio)
3           AD102  Graphic Design and Moving Image Design (portfo...
4           AD103  Textile & Surface Design and Jewellery & Objec...
...           ...                                                ...
1459        WD208                          Manufacturing Engineering
1460        WD210                       Software Systems Development
1461        WD211                                 Creative Computing
1462        WD212                    Recreation and Sport Management
1463        WD230           Mechanical and Manufacturing Engineering

[1464 rows x 2 columns]

In [27]:
courses2020 = df2020[['COURSE CODE2', 'COURSE TITLE']]
courses2020.columns = ['code', 'title']
courses2020

code                                              title
0     AC120                             International Business
1     AC137                                       Liberal Arts
2     AD101   First Year Art & Design (Common Entry,portfolio)
3     AD102  Graphic Design and Moving Image Design (portfo...
4     AD103  Textile & Surface Design and Jewellery & Objec...
...     ...                                                ...
1459  WD208                          Manufacturing Engineering
1460  WD210                       Software Systems Development
1461  WD211                                 Creative Computing
1462  WD212                    Recreation and Sport Management
1463  WD230           Mechanical and Manufacturing Engineering

[1464 rows x 2 columns]

In [28]:
allcourses = pd.concat([courses2021, courses2020])
allcourses

code                                              title
0     AL801     Software Design for Virtual Reality and Gaming
1     AL802  Software Design in Artificial Intelligence for...
2     AL803  Software Design for Mobile Apps and Connected ...
3     AL805    Computer Engineering for Network Infrastructure
4     AL810                                 Quantity Surveying
...     ...                                                ...
1459  WD208                          Manufacturing Engineering
1460  WD210                       Software Systems Development
1461  WD211                                 Creative Computing
1462  WD212                    Recreation and Sport Management
1463  WD230           Mechanical and Manufacturing Engineering

[2413 rows x 2 columns]

In [29]:
allcourses[allcourses.duplicated()]

code                                              title
0     AC120                             International Business
1     AC137                                       Liberal Arts
3     AD102  Graphic Design and Moving Image Design (portfo...
6     AD204                               Fine Art (portfolio)
7     AD211                         Fashion Design (portfolio)
...     ...                                                ...
1455  WD200                                     Arts (options)
1460  WD210                       Software Systems Development
1461  WD211                                 Creative Computing
1462  WD212                    Recreation and Sport Management
1463  WD230           Mechanical and Manufacturing Engineering

[739 rows x 2 columns]

In [30]:
# returns a copy of the dataframe with the duplicates removedb
allcourses.drop_duplicates()

code                                              title
0     AL801     Software Design for Virtual Reality and Gaming
1     AL802  Software Design in Artificial Intelligence for...
2     AL803  Software Design for Mobile Apps and Connected ...
3     AL805    Computer Engineering for Network Infrastructure
4     AL810                                 Quantity Surveying
...     ...                                                ...
1454  WD195  Architectural and Building Information Modelli...
1456  WD205   Molecular Biology with Biopharmaceutical Science
1457  WD206                             Electronic Engineering
1458  WD207                             Mechanical Engineering
1459  WD208                          Manufacturing Engineering

[1674 rows x 2 columns]

In [31]:
#Find all extra copies of duplicate rows
allcourses[allcourses.duplicated(subset=['code'])]

code                                              title
0     AC120                             International Business
1     AC137                                       Liberal Arts
2     AD101   First Year Art & Design (Common Entry,portfolio)
3     AD102  Graphic Design and Moving Image Design (portfo...
4     AD103  Textile & Surface Design and Jewellery & Objec...
...     ...                                                ...
1455  WD200                                     Arts (options)
1460  WD210                       Software Systems Development
1461  WD211                                 Creative Computing
1462  WD212                    Recreation and Sport Management
1463  WD230           Mechanical and Manufacturing Engineering

[896 rows x 2 columns]

In [33]:
#Returns a copy of the data frame with duplicates removed - based only o cade
allcourses.drop_duplicates(subset=['code'], inplace=True)

</br>

## Join to the points

***

In [34]:
# Set the index to the code column.
df2021.set_index('code', inplace=True)
df2021.columns = ['title', 'points_r1_2021', 'points_r2_2021']
df2021

title points_r1_2021  \
code                                                                      
AL801     Software Design for Virtual Reality and Gaming            300   
AL802  Software Design in Artificial Intelligence for...            313   
AL803  Software Design for Mobile Apps and Connected ...            350   
AL805    Computer Engineering for Network Infrastructure            321   
AL810                                 Quantity Surveying            328   
...                                                  ...            ...   
WD211                                 Creative Computing            270   
WD212                    Recreation and Sport Management            262   
WD230           Mechanical and Manufacturing Engineering            230   
WD231                 Early Childhood Care and Education            266   
WD232                       Business Information Systems            261   

      points_r2_2021  
code                  
AL801            NaN  
AL802            NaN  
AL803            NaN  
AL805            NaN  
AL810            NaN  
...              ...  
WD211            NaN  
WD212            NaN  
WD230            230  
WD231            NaN  
WD232            261  

[949 rows x 3 columns]

In [35]:
# Set the index to the code column.
allcourses.set_index('code', inplace=True)

In [36]:
allcourses = allcourses.join(df2021[['points_r1_2021']])
allcourses

title points_r1_2021
code                                                                   
AL801     Software Design for Virtual Reality and Gaming            300
AL802  Software Design in Artificial Intelligence for...            313
AL803  Software Design for Mobile Apps and Connected ...            350
AL805    Computer Engineering for Network Infrastructure            321
AL810                                 Quantity Surveying            328
...                                                  ...            ...
WD188                                Applied Health Care            NaN
WD205   Molecular Biology with Biopharmaceutical Science            NaN
WD206                             Electronic Engineering            NaN
WD207                             Mechanical Engineering            NaN
WD208                          Manufacturing Engineering            NaN

[1517 rows x 2 columns]

In [37]:
df2020_r1 = df2020[['COURSE CODE2', 'R1 POINTS']]
df2020_r1.columns = ['code', 'points_r1_2020']
df2020_r1

code points_r1_2020
0     AC120            209
1     AC137            252
2     AD101       #+matric
3     AD102       #+matric
4     AD103       #+matric
...     ...            ...
1459  WD208            188
1460  WD210            279
1461  WD211            271
1462  WD212            270
1463  WD230            253

[1464 rows x 2 columns]

In [38]:
# Set the index to the code column.
df2020_r1.set_index('code', inplace=True)
df2020_r1

points_r1_2020
code                
AC120            209
AC137            252
AD101       #+matric
AD102       #+matric
AD103       #+matric
...              ...
WD208            188
WD210            279
WD211            271
WD212            270
WD230            253

[1464 rows x 1 columns]

In [39]:
# Join 2020 points to allcourses.
allcourses = allcourses.join(df2020_r1)
allcourses

title points_r1_2021  \
code                                                                      
AL801     Software Design for Virtual Reality and Gaming            300   
AL802  Software Design in Artificial Intelligence for...            313   
AL803  Software Design for Mobile Apps and Connected ...            350   
AL805    Computer Engineering for Network Infrastructure            321   
AL810                                 Quantity Surveying            328   
...                                                  ...            ...   
WD188                                Applied Health Care            NaN   
WD205   Molecular Biology with Biopharmaceutical Science            NaN   
WD206                             Electronic Engineering            NaN   
WD207                             Mechanical Engineering            NaN   
WD208                          Manufacturing Engineering            NaN   

      points_r1_2020  
code                  
AL801            303  
AL802            332  
AL803            337  
AL805            333  
AL810            319  
...              ...  
WD188            201  
WD205            228  
WD206            179  
WD207            198  
WD208            188  

[1517 rows x 3 columns]